# RAG PIPELINE WITH VECTOR DATABASE

### Step 1: Loading the data 

In [10]:
# 1.Loading from a text file
from langchain_community.document_loaders import TextLoader
text=TextLoader('info.txt')
text.load()

[Document(metadata={'source': 'info.txt'}, page_content="\nBest Marketing Psychology Books\nTo study marketing psychology, several books are highly recommended. Here are some of the best ones:\n\nInfluence: The Psychology of Persuasion by Robert B. Cialdini: A classic in the field, this book explores the six universal principles of persuasion that drive human behavior.\nPredictably Irrational by Dan Ariely: This book delves into the hidden forces that shape our decisions, providing insights into consumer behavior and marketing strategies.\nThinking, Fast and Slow by Daniel Kahneman: A foundational book on cognitive bias and prospect theory, it explains how human decision-making is made up of two distinct systems of thinking.\nHooked by Nir Eyal: This book provides a framework for creating habit-forming products and understanding the psychology behind consumer engagement.\nBuyology by Martin Lindstrom: A comprehensive guide to the subconscious mind and its role in consumer decision-maki

In [ ]:
# 2.Loading from a web source (web based loaders)
from dotenv import load_dotenv
import os
load_dotenv()
os.environ['LANGSMITH_API_KEY'] = os.getenv('LANGSMITH_API_KEY')
os.environ['LANGSMITH_TRACING'] = os.getenv('LANGSMITH_TRACING')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')


In [36]:
from langchain_community.document_loaders import WebBaseLoader
import bs4
loader=WebBaseLoader('https://feinternational.com/blog/marketing-psychology-9-strategies-influence-consumers/', 
                    bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("px-6 mx-auto pt-14 pb-40 max-w-[950px]"))))
docs=loader.load()
docs

[Document(metadata={'source': 'https://feinternational.com/blog/marketing-psychology-9-strategies-influence-consumers/'}, page_content='Marketing Psychology: 10 Strategies to Influence Consumers in 2021Thomas SmaleJanuary 1, 2021Building a BusinessFE International BlogThis post was updated on 1 January 2021.\nTo learn about marketing psychology, all marketers should read David McRaney’s You Are Not So Smart. In this book, McRaney highlights the ways in which we’re wired into thinking we are rationale individuals while we are really falling prey to the whims of our psychology. The is also true for marketing psychology and we have put this guide together on nine ways marketers can use this phenomenon to influence consumer behavior.\n1. Reciprocity Principle\nThe idea of reciprocity in social psychology is that if I do something positive for you, you will in turn feel more cooperative and it is predicted that you are more likely to do something positive for me. We can’t help it and it’s a

In [14]:
# Loading a pdf
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('JD.pdf')
pdf=loader.load()
pdf

[Document(metadata={'source': 'JD.pdf', 'page': 0, 'page_label': '1'}, page_content='Data Scientist \nLocation: Remote \nEmployment Type: Full Time \nDepartment: Engineering \nReports To: CTO \n \nAbout Us - https://hub.xyz/ \nHub is at the forefront of transforming how creators and advertisers connect. By combining AI-driven \ninsights with the dynamic possibilities of the creator economy, we’re building a platform that \nempowers creators and gives advertisers unmatched transparency and ROI. \nWith groundbreaking features like Squad key trading and AI-based engagement matching, Hub is \nredefining what’s possible in Web3. Our mission is clear: to unlock the full potential of creators while \nsetting new standards for innovation in the industry. \nBut we’re facing an exhilarating challenge that we need your expertise to solve: \nNow —> How do we accurately and dynamically price creators? \nThe Problem \nCreators are the backbone of Hub, and their value is as unique as their individual

## Step 2: Splitting the data into text chunks

In [37]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter= RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunk_documents=splitter.split_documents(docs)
chunk_documents

[Document(metadata={'source': 'https://feinternational.com/blog/marketing-psychology-9-strategies-influence-consumers/'}, page_content='Marketing Psychology: 10 Strategies to Influence Consumers in 2021Thomas SmaleJanuary 1, 2021Building a BusinessFE International BlogThis post was updated on 1 January 2021.\nTo learn about marketing psychology, all marketers should read David McRaney’s You Are Not So Smart. In this book, McRaney highlights the ways in which we’re wired into thinking we are rationale individuals while we are really falling prey to the whims of our psychology. The is also true for marketing psychology and we have put this guide together on nine ways marketers can use this phenomenon to influence consumer behavior.\n1. Reciprocity Principle\nThe idea of reciprocity in social psychology is that if I do something positive for you, you will in turn feel more cooperative and it is predicted that you are more likely to do something positive for me. We can’t help it and it’s a

## Step 3: Creating vector embeddings of the text chunks and stroing it in the form of a vector database

In [49]:
# Vector Embeddings and vector search
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Weaviate
from langchain_community.vectorstores import FAISS
# from langchain_community.vectorstores import FAISS

db=FAISS.from_documents(chunk_documents,OllamaEmbeddings(model='deepseek-r1:1.5b'))
db

In [43]:
# Performing similarity search on the database
query="Color has a huge impact on our behaviour and should never be underestimated in the context of marketing"
retrieved_result=db.similarity_search(query)
retrieved_result[0].page_content

'In the world of business and commerce, Nudge can be seen in a variety of ways. For example, at restaurants or takeouts, you tend to see one option which is significantly more expensive than anything else. The owners are not expecting many customers to buy this particular item but instead make the second most expensive option seem like a bargain.\nAnother instance can be seen here:'

## Step 4: Retriever and Chain using LangChain

In [54]:
# Design ChatPrompt Template
#Use this website link to get free templates https://promptmatic.ai/prompts 

from langchain.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template("""
Answer the following question based on the context provided.
Think step-by-step before providing a detailed answer

<context>                                        
{context}
</context>

Question: {input}
""")

In [55]:
from langchain_ollama import OllamaLLM
llm=OllamaLLM(model="deepseek-r1:1.5b")
llm

OllamaLLM(model='deepseek-r1:1.5b')

In [56]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain= create_stuff_documents_chain(llm, prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nAnswer the following question based on the context provided.\nThink step-by-step before providing a detailed answer\n\n<context>                                        \n{context}\n</context>\n\nQuestion: {input}\n'), additional_kwargs={})])
| OllamaLLM(model='deepseek-r1:1.5b')
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [57]:
# Using Retriever and Chain -Retrievers are used when you have to perform similarity search on the database through the chain
# On the other hand, you can use Agents when you have multiple sources of loading information and can't define a chain because the information could be present in any of these sources
"""
A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever 
does not need to be able to store documents, only to return (or retrieve) them. Retrievers can be created from vector stores

"""

retriever= db.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x118ba56d0>, search_kwargs={})

In [58]:
# Making a retrieval chain
"""
Retrieval Chain: This chain takes in a user enquiry, which is then passed to the retriever to fetch thre relevant documents.
Those documents and the original input are hence passed to the LLM to get a response

"""
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [61]:
response=retrieval_chain.invoke({"input":"Reciprocity Principle"})

In [62]:
response["context"][0].page_content

'This is a promotion for LTV Conference for people in the SaaS industry. It’s an excellent example of enticing potential buyers to make their purchase earlier, with a focus on the reward of saving 75% on the price. Even the code plays a part, promoting the idea of “LTV friends,” which lends a personal touch.\nHaving a deep understanding of marketing psychology can make or break your business. Take the choice study mentioned earlier. If you could increase your conversion rate from 3% to 30% just by reducing the options you give customers, what could that do for your business? The key takeaway here is to be continually testing, monitoring results and optimizing. As this article has shown, small tweaks can make a huge difference.\nFurther Reading:\n15 Psychological Studies That Will Boost Your Social media Marketing\n39 Resources for Understanding the Science & Psychology Behind Great Marketing\nThe Complete Guide to Understanding Consumer Psychology'